In [1]:
import tensair_aux as tensair

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.python.keras.engine import data_adapter
from tensorflow.python.eager import backprop
import os

## Define TensorFlow Model

One may create a TensorFlow model as usual but extending TensAIR instead of Model.

In [2]:
class Cifar(tensair.TensAIR):
    def __init__(self):
        super(Cifar, self).__init__()
        self.l1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3))
        self.l2 = layers.MaxPooling2D((2, 2))
        self.l3 = layers.Conv2D(64, (3, 3), activation='relu')
        self.l4 = layers.MaxPooling2D((2, 2))
        self.l5 = layers.Conv2D(64, (3, 3), activation='relu')
        self.l6 = layers.Flatten()
        self.l7 = layers.Dense(64, activation='relu')
        self.l8 = layers.Dense(10)
        
    @tf.function
    def call(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        x = self.l6(x)
        x = self.l7(x)
        x = self.l8(x)
        return x
    
    @tf.function
    def freeze_layers(self):
        for layer in self.layers:
            layer.trainable = False
    
        # Unfreeze the last two layers
        self.l7.trainable = True
        self.l8.trainable = True

In [3]:
#compile TensorFlow model as usual
model = Cifar()

learning_rate=0.01
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam()

model.compile(optimizer=opt,
              loss=loss,
              metrics=['accuracy'])

# Pre-train model

In this example, we pre-train the model using CIFAR-10 for 20 epochs

In [4]:
from tensorflow.keras import datasets

In [5]:
#Download Cifar dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [6]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_data=(test_images, test_labels))

Epoch 1/20


2024-03-06 18:52:15.862599: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


391/391 [==============================] - 10s 24ms/step - loss: 1.6725 - accuracy: 0.3829 - val_loss: 1.4169 - val_accuracy: 0.4831
Epoch 2/20
391/391 [==============================] - 9s 24ms/step - loss: 1.3330 - accuracy: 0.5219 - val_loss: 1.2632 - val_accuracy: 0.5483
Epoch 3/20
391/391 [==============================] - 10s 25ms/step - loss: 1.1976 - accuracy: 0.5731 - val_loss: 1.1919 - val_accuracy: 0.5804
Epoch 4/20
391/391 [==============================] - 10s 26ms/step - loss: 1.1037 - accuracy: 0.6099 - val_loss: 1.0553 - val_accuracy: 0.6272
Epoch 5/20
391/391 [==============================] - 11s 29ms/step - loss: 1.0153 - accuracy: 0.6417 - val_loss: 1.0228 - val_accuracy: 0.6374
Epoch 6/20
391/391 [==============================] - 10s 26ms/step - loss: 0.9636 - accuracy: 0.6612 - val_loss: 0.9811 - val_accuracy: 0.6575
Epoch 7/20
391/391 [==============================] - 10s 26ms/step - loss: 0.9126 - accuracy: 0.6822 - val_loss: 0.9558 - val_accuracy: 0.6710
Epoc

# Prepare model for deployment

To prepare the model for deployment, we fine tune only its last two layers in real-time. Thus, we freeze the first layers (optional).

Then, we re-compile the model using SGD as optimizer.

We later initialize the TF graph making a test prediction.

At last, we initialize the gradients that will be broadcasted via TensAIR (mandatory)

In [7]:
model.freeze_layers()

In [8]:
loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.SGD()

model.compile(optimizer=opt,
              loss=loss,
              metrics=['accuracy'])

In [9]:
x = tf.zeros((1, 32, 32, 3), dtype=float)
model.predict(x)

1/1 [==============================] - 0s 59ms/step


array([[-1.1212026, -2.744787 , -0.92356  ,  1.4073707,  2.6265948,
        -2.3970294, -1.9702166, -3.2693338,  1.8532662, -2.4577236]],
      dtype=float32)

In [10]:
model.init_delta(model.trainable_weights)

# Save Model

To save the model, we simply define the dimentions and types of the input tensors used during training, a pass those (along with the model) to the tensair.define_signatures function) to obtain the signature of the functions

At last, we save the model on the desired location uwing the signatures previously obtained

In [11]:
input_tensors_dims = [[None,32,32,3],[None]]
input_tensors_types = [tf.float32,tf.int32]
input_tensors_structure = (1,1)

In [12]:
signatures = tensair.define_signatures(model, input_tensors_dims,input_tensors_types,input_tensors_structure)

In [13]:
tensair_path = os.environ.get("TENSAIR_PATH")

In [14]:
model.save(tensair_path+ "/data/demo/cifar_model_demo.tf", save_format="tf", signatures=signatures)

INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/demo/cifar_model_demo.tf/assets


INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/demo/cifar_model_demo.tf/assets
